In [281]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import math
from statistics import mode
import tensorflow as tf
import random
import os

In [282]:
# Read the CSV file to classify whether the student has a A score 
df_matA = pd.read_csv('mat_A.csv', sep=',')
data = df_matA.to_numpy()[:, 1:]

# standarize the data 
indices_to_standarize = []
for i, feature in enumerate(data[0]):
    if type(feature) != str and i != (data.shape[1]-1):
        indices_to_standarize.append(i)

for i in indices_to_standarize:
    m = np.mean(data[:, i])
    std = np.std(data[:, i])
    data[:, i] = (data[:, i]-m)/std

# split in to training and testing
X_train, X_test, y_train, y_test = train_test_split(data[:, :-1], data[:, -1], test_size=0.1, random_state=0)

In [283]:
def dist(x1, x2):
    total_dist = 0
    
    for i, feature in enumerate(x1):
        # for numerical values, the distance is manhattan
        if type(feature) != str:
            total_dist += abs(x1[i] - x2[i])
        # for categorical values, the distance is hamming
        else:
            if x1[i] != x2[i]:
                total_dist += 1
    return total_dist

In [284]:
def find_min(dists, k):
    x_sorted = np.sort(dists)
    min_values = x_sorted[:k]
    indices = [dists.index(value) for value in min_values]
    return indices
    

In [285]:
# knn
def knn(X_train, X_test, y_train, y_test, k):

    y_pred = []

    # for each test data
    for x in X_test:
        
        # get an array represent the distance between x and each training data
        distances = []
        for x_train in X_train:
            distances.append(dist(x, x_train))
        
        # find the k lowest distance
        lowest_indices = find_min(distances, k)
        
        # majority vote from these k
        pred = mode(y_train[lowest_indices])
        
        # append the predicted result to the prediction array
        y_pred.append(pred)
        
    accuracy = sum(y_pred == y_test)/len(y_pred)
    return accuracy

In [286]:
# ANN
def ann(data, act_func1, act_func2, act_func3):
    random_seed = 0
    os.environ['PYTHONHASHSEED']=str(random_seed)
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)
    
    # process data by change yes/no to 0/1 and delete all categorical data:
    new_data = data.copy()
    columns_to_delete = []
    for sample in new_data:
        for i in range(len(sample)):
            if sample[i] == 'yes':
                sample[i] = 1
            elif sample[i] == 'no':
                sample[i] = 0
    for i in range(len(new_data[0])):
        if type(new_data[i][i]) == str:
            columns_to_delete.append(i)

    new_data = np.delete(new_data, columns_to_delete, axis = 1)
    # split in to training and testing
    X_train, X_test, y_train, y_test = train_test_split(new_data[:, :-1], new_data[:, -1], test_size=0.1, random_state=0)

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)
    y_train = y_train.astype(np.float32)
    y_test = y_test.astype(np.float32)

    # Initialize the ANN model
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=64, activation=act_func1))
    model.add(tf.keras.layers.Dense(units=32, activation=act_func2))
    model.add(tf.keras.layers.Dense(units=1, activation=act_func3))

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, batch_size=32, epochs=100)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    return accuracy

In [287]:
print(knn(X_train, X_test, y_train, y_test, 6))
print(ann(data, 'relu', 'relu', 'sigmoid'))

0.9
Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 736us/step - accuracy: 0.4867 - loss: 0.7149
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 557us/step - accuracy: 0.8048 - loss: 0.5567
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step - accuracy: 0.8348 - loss: 0.4719
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 664us/step - accuracy: 0.8678 - loss: 0.3995
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 639us/step - accuracy: 0.9009 - loss: 0.3340
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 540us/step - accuracy: 0.9181 - loss: 0.2813
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 575us/step - accuracy: 0.9180 - loss: 0.2410
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 507us/step - accuracy: 0.9302 - loss: 0.2112
Epoch 9/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 463us/step - accuracy: 0.9324 - loss: 0.1875
Epoch 10/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 489us/step - accuracy: 0.9377 - loss: 0.1690
Epoch 11/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 481us/step - accuracy: 0.9427 - loss: 0.1537
Epoch 12/100
12/12 ━━━━━━━━━━━━━━━━━━